In [ ]:
!pip install -q transformers sentence-transformers faiss-cpu accelerate

In [ ]:
import transformers
import sentence_transformers
import faiss

print("All libraries installed successfully ✅")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

print("LLM loaded successfully ✅")


In [ ]:
prompt = "Explain RAG in one sentence."

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(
    **inputs,
    max_new_tokens=50
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
import os

os.listdir("/content")

In [ ]:
!pip install -q pypdf

In [ ]:
from pypdf import PdfReader

reader = PdfReader("/content/swiggy_con_call.pdf")

text = ""
for page in reader.pages:
    text += page.extract_text()

print(text[:1000])  # preview first 1000 characters

In [ ]:
def chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0

    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap

    return chunks

In [ ]:
chunks = chunk_text(text)

print(f"Total chunks created: {len(chunks)}")
print("\n--- Sample chunk ---\n")
print(chunks[0][:800])


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

print("Embedding model loaded ✅")

In [ ]:
chunk_embeddings = embedding_model.encode(chunks, show_progress_bar=True)

print(f"Embedding shape: {chunk_embeddings.shape}")

In [ ]:
import faiss
import numpy as np

dimension = chunk_embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)
index.add(np.array(chunk_embeddings))

print(f"FAISS index created with {index.ntotal} vectors ✅")


In [ ]:
# ---- Document Scope Configuration ----
DOCUMENT_SCOPE = {
    "company": "Swiggy",
    "blocked_entities": ["zomato", "ubereats", "blinkit"]
}

In [ ]:
def retrieve_chunks_with_scores(query, k=3):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(query_embedding, k)
    return distances[0], [chunks[i] for i in indices[0]]

In [ ]:
query = "What did Swiggy say about profitability?"
results = retrieve_chunks(query)

print("\n--- Retrieved Chunks ---\n")
for i, chunk in enumerate(results):
    print(f"Chunk {i+1}:\n{chunk[:500]}\n")


In [ ]:
print(chunk)

In [ ]:
def build_prompt(context_chunks, question):
    context = "\n\n".join(context_chunks)

    prompt = f"""
You are an analyst assistant.
Answer the question ONLY using the context below.
If the answer is not present, say "Not mentioned in the document".

Context:
{context}

Question:
{question}

Answer:
"""
    return prompt

In [ ]:
def rag_answer(question, k=3, relevance_threshold=1.2):

    q_lower = question.lower()

    # 🚨 HARD DOCUMENT SCOPE CHECK
    for entity in DOCUMENT_SCOPE["blocked_entities"]:
        if entity in q_lower:
            return (
                f"This document is about {DOCUMENT_SCOPE['company']}. "
                f"It does not contain information about {entity.capitalize()}."
            )

    # 🔍 Semantic retrieval
    distances, context_chunks = retrieve_chunks_with_scores(question, k)

    # 🚨 Retrieval confidence gate
    if min(distances) > relevance_threshold:
        return "This document does not contain information related to this question."

    context = "\n\n".join(context_chunks)

    prompt = f"""
You are a document-grounded assistant.

STRICT RULES:
- Use ONLY the context below
- Do NOT use external knowledge
- Do NOT infer or guess
- If the answer is not explicitly present, say:
  "This document does not contain information related to this question."

Context:
{context}

Question:
{question}

Answer:
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=2048
    )

    outputs = model.generate(
        **inputs,
        max_new_tokens=120,
        temperature=0.1
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
question = "What did Swiggy management say about profitability and margins?"
answer = rag_answer(question)

print(answer)


In [ ]:
question = "Give a 200 word summary of Swiggy's food delivery business."

answer = rag_answer(question, k=5)

print(answer)

In [ ]:
answer = rag_answer(
    "Give a 200 word summary of Swiggy's food delivery business.",
    k=5
)

# Clean and print one line below the other
sentences = answer.replace("\n", " ").split(".")

print("\n--- Clean Line-by-Line Output ---\n")
for s in sentences:
    s = s.strip()
    if len(s) > 20:  # avoid very small junk lines
        print("- " + s + ".")

In [ ]:
!pip install -q gradio

In [ ]:
def rag_ui(question):
    if question.strip() == "":
        return "Please enter a question."

    answer = rag_answer(question, k=5)

    # Clean formatting for UI
    sentences = answer.replace("\n", " ").split(".")
    formatted = ""
    for s in sentences:
        s = s.strip()
        if len(s) > 20:
            formatted += "• " + s + ".\n\n"

    return formatted

In [ ]:
import gradio as gr

ui = gr.Interface(
    fn=rag_ui,
    inputs=gr.Textbox(
        lines=3,
        placeholder="Ask about Swiggy food delivery business, profitability, growth..."
    ),
    outputs=gr.Textbox(
        lines=15,
        label="RAG Answer"
    ),
    title="📊 Swiggy Earnings Call RAG Assistant",
    description="Ask questions grounded ONLY in the Swiggy earnings call document."
)

ui.launch()


In [46]:
!ls

data  notebooks  sample_data  src  swiggy_con_call.pdf


In [49]:
!ls notebooks
!ls data
!sed -n '1,20p' README.md

sed: can't read README.md: No such file or directory


In [50]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [51]:
!git commit -m "Initial commit: end-to-end RAG system with hallucination guardrails"


Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@06990d43c785.(none)')


In [ ]:
#Tools used
#Read pdf: pypdf
#Vector database: FAISS
#Chuncking: RecursiveCharacterTextSplitter
#Ebidding:
Tool used: sentence-transformers
Model: all-MiniLM-L6-v2
#Query Embedding & Retrieval: sentence-transformers + FAISS
#UI layer: Gradio
